Imagine a game where one or more rats can attack a player. Each individual rat has an initial attack value of 1. However, rats attack as a swarm, so each rat's attack value is actually equal to the total number of rats in play.

Given that a rat enters play through the initializer and leaves play (dies) via its __exit__ method, please implement the Game and Rat classes so that, at any point in the game, the Attack value of a rat is always consistent.

Here's a sample unit test your code should pass:

    def test_three_rats_one_dies(self):
        game = Game()
     
        rat = Rat(game)
        self.assertEqual(1, rat.attack)
     
        rat2 = Rat(game)
        self.assertEqual(2, rat.attack)
        self.assertEqual(2, rat2.attack)
     
        with Rat(game) as rat3:
            self.assertEqual(3, rat.attack)
            self.assertEqual(3, rat2.attack)
            self.assertEqual(3, rat3.attack)
     
        self.assertEqual(2, rat.attack)
        self.assertEqual(2, rat2.attack)

In [34]:
from typing import Any


class Event(list):
    def __call__(self, *args: Any, **kwargs: Any) -> Any:
        for e in self:
            e(*args, **kwargs)
    
class Game:
    def __init__(self):
        self.alive_rats = Event()
        self.notify_rats = Event()
        self.dead_rats = Event()

    def new_rat(self, rat):
        self.alive_rats(rat)

    def dead_rat(self):
        self.dead_rats()

    def notify_rat(self, rat):
        self.notify_rats(rat)

class Rat:
    def __init__(self, game: Game):
        self.game = game
        self.attack = 1
        self.game.alive_rats.append(self.increase_attack)
        self.game.dead_rats.append(self.decrease_attack)
        self.game.notify_rats.append(self.notify_rat)
        self.game.new_rat(self)

    def increase_attack(self, sender):
        if sender is not self:
            self.attack += 1
            self.game.notify_rat(sender)
    
    def decrease_attack(self):
        self.attack -= 1

    def notify_rat(self, sender):
        if sender is self:
            self.attack += 1

    def __enter__(self):
        return self
    
    def __exit__(self, *args, **kwargs):
        self.game.dead_rat()

In [35]:
game = Game()
r1 = Rat(game)
r2 = Rat(game)
print(r2.attack)
r3 = Rat(game)

with Rat(game) as r4:
    print(r1.attack)

print(r1.attack)

2
4
3


In [36]:
from unittest import TestCase

class Tests(TestCase):
    def test_three_rats_one_dies(self):
        game = Game()

        rat = Rat(game)
        self.assertEqual(1, rat.attack)

        rat2 = Rat(game)
        self.assertEqual(2, rat.attack)
        self.assertEqual(2, rat2.attack)

        with Rat(game) as rat3:
            self.assertEqual(3, rat.attack)
            self.assertEqual(3, rat2.attack)
            self.assertEqual(3, rat3.attack)

        self.assertEqual(2, rat.attack)
        self.assertEqual(2, rat2.attack)

t = Tests()
t.test_three_rats_one_dies()